In [1]:
from components.fuzzymatching import FuzzyMatchingFactory


config = {'csv_path': './backend/db/keyframes.csv'}
ocr_matcher = FuzzyMatchingFactory.produce('rapidwuzzy', **config)

keyword = "Apple"
result = ocr_matcher.run(keyword)

/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore
/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initializatio

In [2]:
keyword = "Apple"
result = ocr_matcher.run(keyword)

In [3]:
img_paths = ocr_matcher.get_image_paths(result)
img_paths

['/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L07_V011-12586.webp',
 '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L18_V022-30924.webp',
 '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L22_V010-00051.webp',
 '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L19_V030-17204.webp',
 '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V017-00435.webp',
 '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V045-00668.webp',
 '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L10_V024-21236.webp',
 '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L02_V013-10274.webp',
 '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L09_V010-00657.webp',
 '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L09_V015-36400.webp']

In [4]:
from components.llms import AgentFactory
from components.llms.prompts import EXTRACT_KEYWORDS

import os 
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm_config = {'api_key': GOOGLE_API_KEY}
llm_agent = AgentFactory.produce(provider='gemini', 
                                 **llm_config)
llm_agent

In [5]:
from utils.pre_processing import clean
from components.translation import TranslatorFactory


translator = TranslatorFactory.produce(provider='googletrans')


In [6]:
query = "Đoạn giới thiệu về lễ hội Việt - Nhật. Trong lễ hội có các phân cảnh những chiếc lồng đèn kiểu Nhật cùng những chiếc dù Wagasa. Phân đoạn giữa có hình ảnh của chú mèo máy Doraemon khổng lồ và có những người xung quanh chụp ảnh với nó. Phân đoạn sau đó có hình ảnh một ban nhạc Nhật chơi các nhạc cụ truyền thống trên sân khấu. Xuyên suốt đoạn giới thiệu là một số đoạn phỏng vấn những người tham gia lễ hội. Hỏi có tổng cộng bao nhiêu người khác nhau được phỏng vấn trong đoạn giới thiệu trên?"
translated_query = translator.run(query)
translated_query

'The introduction of Vietnam - Japan Festival.In the festival there are scenes of Japanese -style lanterns and Wagasa umbrellas.The middle segment has the image of the giant Doraemon cat and there are people around him to take photos with it.The segment then has a picture of a Japanese band playing traditional musical instruments on stage.Throughout the introduction are some interviews with the festival participants.Ask how many different people are interviewed in the introduction above?'

In [7]:
clean_query = clean(translated_query)
clean_query

'the introduction of vietnam - japan festival.in the festival there are scenes of japanese -style lanterns and wagasa umbrellas.the middle segment has the image of the giant doraemon cat and there are people around him to take photos with it.the segment then has a picture of a japanese band playing traditional musical instruments on stage.throughout the introduction are some interviews with the festival participants.ask how many different people are interviewed in the introduction above?'

In [26]:
import json

kw_v1 = llm_agent.run(EXTRACT_KEYWORDS + '\n' + translated_query)
kw_v1 = json.loads(kw_v1)
kw_v1 = [d['keyword'] for d in kw_v1]
kw_v1

['Vietnam - Japan Festival',
 'Japanese -style lanterns',
 'Wagasa umbrellas',
 'Doraemon',
 'Japanese band',
 'traditional musical instruments']

In [27]:
import json

kw_v2 = llm_agent.run(EXTRACT_KEYWORDS + '\n' + clean_query)
print(kw_v2)
print(type(kw_v2))
kw_v2 = json.loads(kw_v2)
kw_v2 = [d['keyword'] for d in kw_v2]
kw_v2

[{"keyword": "Vietnam-Japan Festival"}, {"keyword": "Japanese lanterns"}, {"keyword": "wagasa umbrellas"}, {"keyword": "Doraemon"}, {"keyword": "Japanese band"}, {"keyword": "traditional musical instruments"}, {"keyword": "festival participants"}]

<class 'str'>


['Vietnam-Japan Festival',
 'Japanese lanterns',
 'wagasa umbrellas',
 'Doraemon',
 'Japanese band',
 'traditional musical instruments',
 'festival participants']

In [28]:
import json

kw_v3 = llm_agent.run(EXTRACT_KEYWORDS + '\n' + query)
kw_v3 = json.loads(kw_v3)
kw_v3 = [d['keyword'] for d in kw_v3]
kw_v3

['Việt - Nhật',
 'lễ hội',
 'lồng đèn',
 'Wagasa',
 'Doraemon',
 'nhạc cụ truyền thống',
 'phỏng vấn']

In [31]:
matching_paths_v1 = []

for kw in kw_v1: 
    result = ocr_matcher.run(kw)
    img_paths = ocr_matcher.get_image_paths(result)
    matching_paths_v1 += img_paths

print(len(matching_paths_v1))
matching_paths_v1 = list(set(matching_paths_v1))
print(len(matching_paths_v1))
print(matching_paths_v1)

60
53
['/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V043-10716.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V018-00000.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L17_V009-00042.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V007-00984.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V005-01531.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L20_V019-24400.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L11_V006-12142.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L18_V008-00318.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V004-06871.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V029-16206.webp', '/home/octoop

In [33]:
matching_paths_v2 = []

for kw in kw_v2: 
    result = ocr_matcher.run(kw)
    img_paths = ocr_matcher.get_image_paths(result)
    matching_paths_v2 += img_paths

print(len(matching_paths_v2))
matching_paths_v2 = list(set(matching_paths_v2))
print(len(matching_paths_v2))
print(matching_paths_v2)

70
57
['/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V043-10716.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V018-00000.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L17_V009-00042.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V007-00984.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V005-01531.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V004-08976.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L01_V031-20472.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L20_V019-24400.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L11_V006-12142.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L01_V031-21436.webp', '/home/octoop

In [34]:
matching_paths_v3 = []

for kw in kw_v3: 
    result = ocr_matcher.run(kw)
    img_paths = ocr_matcher.get_image_paths(result)
    matching_paths_v3 += img_paths

print(len(matching_paths_v3))
matching_paths_v3 = list(set(matching_paths_v3))
print(len(matching_paths_v3))
print(matching_paths_v3)

70
67
['/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L02_V010-12470.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V029-15532.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L20_V031-11320.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L02_V001-06644.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V013-01155.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V015-05316.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L24_V011-02194.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L20_V019-24400.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L04_V005-15958.webp', '/home/octoopt/Desktop/Lab/Personal/evento/backend/db/s_optimized_keyframes/L20_V020-07433.webp', '/home/octoop